In [1]:
from openai import OpenAI
import pandas as pd

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="xxxxx",
)

def answer(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature = 0
    )
    return response.choices[0].message.content.strip()

In [2]:
def feed(i,df):
    text = (
    "Evaluate the sentiment toward cultured meat that this tweet expresses: " + df['Tweet'][i] + '\n'+'Return your response only as positive (which indicates positive sentiment toward cultured meat is expressed), negative (which indicates negative sentiment toward cultured meat is expressed), neutral (which indicates no sentiment toward cultured meat is expressed), or combination (which indicates that both positive and negative sentiments toward cultured meat are expressed).')
    return text

In [3]:
import pandas as pd
Paris_1 = pd.read_csv('Coding_Paris_Week1.csv')
Paris_2 = pd.read_csv('Coding_Paris_Week2.csv')
Paris_3 = pd.read_csv('Coding_Paris_Week3.csv')
Paris_4 = pd.read_csv('Coding_Paris_Week4.csv')
Paris_5 = pd.read_csv('Coding_Paris_Week5.csv')
Shawn_1 = pd.read_csv('Coding_Shawn_Week1.csv')
Shawn_2 = pd.read_csv('Coding_Shawn_Week2.csv')
Shawn_3 = pd.read_csv('Coding_Shawn_Week3.csv')
Shawn_4 = pd.read_csv('Coding_Shawn_Week4.csv')
Shawn_5 = pd.read_csv('Coding_Shawn_Week5.csv')
Tianli_1 = pd.read_csv('Coding_Tianli_Week1.csv')
Tianli_2 = pd.read_csv('Coding_Tianli_Week2.csv')
Tianli_3 = pd.read_csv('Coding_Tianli_Week3.csv')
Tianli_4 = pd.read_csv('Coding_Tianli_Week4.csv')
Tianli_5 = pd.read_csv('Coding_Tianli_Week5.csv')
# Combine all the data frames into one
combined_df = pd.concat([
    Paris_1, Paris_2, Paris_3, Paris_4, Paris_5,
    Shawn_1, Shawn_2, Shawn_3, Shawn_4, Shawn_5,
    Tianli_1, Tianli_2, Tianli_3, Tianli_4, Tianli_5
], ignore_index=True)

combined_df['Sentiment'] = combined_df['Sentiment'].astype(int)

An example of actual prompt:

In [4]:
print(feed(1,combined_df))

Evaluate the sentiment toward cultured meat that this tweet expresses: The future will be full of lab grown meat: http://t.co/xFPgLz0yg2 http://t.co/u1voAdrQ0a http://t.co/yxksEjq3kZ
Return your response only as positive (which indicates positive sentiment toward cultured meat is expressed), negative (which indicates negative sentiment toward cultured meat is expressed), neutral (which indicates no sentiment toward cultured meat is expressed), or combination (which indicates that both positive and negative sentiments toward cultured meat are expressed).


In [5]:
train_df = combined_df.sample(frac=0.8, random_state=2024)

# remaining 20% as validation set
validation_df = combined_df.drop(train_df.index)

In [6]:
validation_df = validation_df.reset_index(drop=True)
validation_df

,Tweet,Sentiment
0,@csimpsyo @Tbogin @jonlovett Cultured meat,3
1,RT @ndonyourtable What's the difference betwee...,3
2,#Technology #Tech Lab-Grown Meat Is Coming htt...,3
3,This year is the first time cultivated meat ha...,3
4,"RT @NewHarvestOrg 🍗and @UmaValeti, who co-foun...",3
...,...,...
4590,@Joseph_Plant What goes into lab grown meat? I...,2
4591,Google Funding Lab Grown Meat… No Animals Kill...,1
4592,RT @Orbyne #LSEForum cultured meat avoids the ...,1
4593,@MusadADroid @AuthorGusPegel The answer would ...,4


In [7]:
import time
sentiment = []
start = time.time()
count = 0
for i, v in validation_df.iterrows():
    sentiment.append(answer(feed(i,validation_df)))
time_used = time.time()-start
time_used

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: xxxxx. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [7]:
len(sentiment)

4595

In [9]:
validation_df['GPT-4o'] = [i for i in sentiment]

In [10]:
validation_df

,Unnamed: 0,Tweet,Sentiment,GPT-4o,GPT-4o_number,GPT-4o_lowercase,GPT-4o_2,GPT-4o_3
0,0,@csimpsyo @Tbogin @jonlovett Cultured meat,3,neutral,3,neutral,Neutral,neutral
1,1,RT @ndonyourtable What's the difference betwee...,3,Neutral,3,neutral,Neutral,Neutral
2,2,#Technology #Tech Lab-Grown Meat Is Coming htt...,3,Neutral,3,neutral,Neutral,Neutral
3,3,This year is the first time cultivated meat ha...,3,neutral,3,neutral,neutral,neutral
4,4,"RT @NewHarvestOrg 🍗and @UmaValeti, who co-foun...",3,positive,1,positive,Positive,Positive
...,...,...,...,...,...,...,...,...
4590,4590,@Joseph_Plant What goes into lab grown meat? I...,2,Negative,2,negative,Negative,Negative
4591,4591,Google Funding Lab Grown Meat… No Animals Kill...,1,positive,1,positive,positive,positive
4592,4592,RT @Orbyne #LSEForum cultured meat avoids the ...,1,positive,1,positive,positive,positive
4593,4593,@MusadADroid @AuthorGusPegel The answer would ...,4,positive,1,positive,positive,positive


In [18]:
validation_df['GPT-4o_number'] = validation_df['GPT-4o_number'].fillna(0).astype(int)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(validation_df['Sentiment'], validation_df['GPT-4o_number'])
precision = precision_score(validation_df['Sentiment'], validation_df['GPT-4o_number'], average='weighted')
recall = recall_score(validation_df['Sentiment'], validation_df['GPT-4o_number'], average='weighted')
f1 = f1_score(validation_df['Sentiment'], validation_df['GPT-4o_number'], average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.7207834602829162
Precision: 0.7615508544676489
Recall: 0.7207834602829162
F1 Score: 0.732738742633678


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
